### Modules and Config

In [15]:
# Installed
import numpy as np
import theano
import theano.tensor as TT
from pdb import set_trace
theano.config.optimizer='None'
theano.gof.cc.get_module_cache().clear()

# Local
import shared
import misc 
reload(misc)
import nef
reload(nef)
import gru
reload(gru)

<module 'gru' from 'gru.py'>

**Configuration**

In [16]:
# outer world
nr_words = 40     # Number of words
voc_size = 100    # Vocabulary size 
emb_size = 50     # Embedding size
nr_classes = 2    # Number of classes
# 
# Sketch matrix
sketch_dim = 10   # Sketch dimension
nr_sketch = 3     # Sketch number
# Attention
attention_dim = 6 # Attention dimension
conv_dim = 3      # Attention convolutionsl dimension 

# Train param
lrate = 1.0

### Forward pass

In [29]:
# Embeddings
E = shared.random(size=(emb_size, voc_size), name='embeddings')
# GRU on the source
param_GRU = gru.init_gru(emb_size, sketch_dim, bias=True)

In [30]:
# Input
x = TT.ivector('x')
# Embedding
emb = E[:, x]
# RNN
H = gru.gru(emb.T, param_GRU).T
# Easy first 
p_y = nef.easy_first(H, nr_sketch, attention_dim, sketch_dim, conv_dim, nr_classes)

In [31]:
param = misc.get_param(p_y, named_only=True)
param

[inference.w_p,
 gru.b_U,
 gru.U,
 gru.b_Ur,
 gru.U_r,
 gru.b_Uz,
 gru.U_z,
 gru.[b b_r b_z],
 gru.[W, W_r, W_z],
 embeddings,
 attention.W_sz,
 attention.W_hz,
 attention.v,
 sketch.W_s,
 sketch.W_ss,
 sketch.W_hs,
 inference.W_sp]

**Fake Example Forward**

In [32]:
# Fake data
data_x = np.random.randint(0, voc_size-1, size=(nr_words)).astype('int32')
dbg = theano.function([x], p_y)
print [dat.shape for dat in dbg(data_x)]

[(40,), (40,)]


### Training

In [33]:
# Cost for ground truth
y = TT.ivector('x')
cost = -TT.mean(TT.log(p_y)[y, TT.arange(y.shape[0])])

**Fake Example Cost**

In [34]:
# Fake data
data_x = np.random.randint(0, voc_size-1, size=(nr_words)).astype('int32')
data_y = np.random.randint(0, nr_classes-1, size=(nr_words)).astype('int32')
dbg = theano.function([x, y], cost)
print dbg(data_x, data_y)

0.693147181993


In [ ]:
updates = [(par, par - lrate*TT.grad(cost, par)) for par in param]
batch_train = theano.function([x, y], cost, updates=updates)

**Fake Example Batch Update**

In [ ]:
batch_train(data_x, data_y)